In [9]:
import pandas as pd
import requests

In [72]:
textile_df = pd.read_csv('../data/Fabric Sustainability Data - Textile Research.csv')
textile_df.head()

,material_id,conventional_material_id,source,preferred,biodegradable,microplastics,deforestation,forest_management,chemical_hazard,closed_loop_process,animal_cruelty,recycled,agricultural_impact,organic
0,COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False
1,RECYCLED_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,True,False,False
2,ORGANIC_COTTON,COTTON,plant,True,True,False,False,False,False,False,False,False,False,True
3,EGYPTIAN_COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False
4,PIMA_COTTON,COTTON,plant,False,True,False,False,False,False,False,False,False,True,False


In [63]:
# item is in this format: item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

def get_material_score(item) -> float:
    # keep only materials and percentages
    materials_and_percents = {(key.replace(' ', '_')).upper(): value for key, value in item.items() if key != 'item'}

    # weights
    material_weight = 1/3
    end_of_life_weight = 1/3
    agr_impact_weight = 1/3

    # variables for percentage sums
    material_percent_sum = 0
    eol_percent_sum = 0
    agr_impact_sum = 0

    # go through each material and its percentage
    for material, percent in materials_and_percents.items():
        
        # convert percentage to decimal
        normalized_percent = percent / 100
        print(material, normalized_percent)

        # get whether the material is "preferred"
        preferred_value = textile_df.loc[textile_df['material_id'] == material, 'preferred'].values[0]
        print(f'preferred: {preferred_value}')

        # if it is, add the material percentage to sum
        if preferred_value:
            material_percent_sum += normalized_percent

        # get whether the material is "biodegradable"
        biodegradable_value = textile_df.loc[textile_df['material_id'] == material, 'biodegradable'].values[0]
        print(f'biodegradable: {biodegradable_value}')

        # if it is, add the material percentage to sum
        if biodegradable_value:
            eol_percent_sum += normalized_percent

        # get whether there are good agricultural practices
        agr_impact_value = textile_df.loc[textile_df['material_id'] == material, 'agricultural_impact'].values[0]
        print(f'agricultural impact: {agr_impact_value}')

        # if it is, add the material percentage to sum
        if not agr_impact_value:
            agr_impact_sum += normalized_percent

        print()

    # get material score after multiplying by weight
    material_score = material_weight * material_percent_sum

    # get end of life score after multiplying by weight
    end_of_life_score = end_of_life_weight * eol_percent_sum

    # get agricultural impact score after multiplying by weight
    agricultural_impact_score = agr_impact_weight * agr_impact_sum

    # get final score out of 5
    final_score = round(5 * (material_score + end_of_life_score + agricultural_impact_score), 2)
    # print(f'final score out of 5: {final_score}')

    return final_score

In [58]:
# example from material_calculator.ipynb
item = {'item': 'Linen Blend Belted Sleeveless Lapel Dress (JW Anderson)', 'silk': 53, 'linen': 33, 'cotton': 14}

# get score out of 5
get_material_score(item)

SILK 0.53
preferred: False
biodegradable: True
agricultural impact: False

LINEN 0.33
preferred: False
biodegradable: True
agricultural impact: False

COTTON 0.14
preferred: False
biodegradable: True
agricultural impact: True



3.1

In [13]:
# getting product rank (out of 1)
url = 'https://api.selflessclothes.com/research/calculator?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14'

response = requests.get(url)
data = response.json()

print(data.get('rank')) # 0.08

# https://www.selflessclothes.com/blog/sustainability-calculator/?material=SILK&material=LINEN&material=COTTON&percentage=53&percentage=33&percentage=14
# note: for these materials, the score out of 5 is 0.4 (0.08 * 5)

0.08
